<a href="https://colab.research.google.com/github/DeepanIsCool/CuraGen2024/blob/main/CuraGenModelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# from google.colab import files
# uploaded = files.upload()

In [29]:
!git clone https://github.com/DeepanIsCool/CuraGen2024.git

Cloning into 'CuraGen2024'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 1), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 77.15 KiB | 1.13 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [30]:
%cd CuraGen2024
!ls

/content/CuraGen2024/CuraGen2024
data


In [31]:
import pandas as pd
data = pd.read_csv('/content/CuraGen2024/data/Disease_symptom_and_patient_profile_dataset.csv')
data.head() # View the first few rows


print(data.info())
print(data['Disease'].value_counts())


X = data.drop('Disease', axis=1)  # Features
y = data['Disease']  # Target

X = pd.get_dummies(X)  # One-hot encode categorical features

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data = data.dropna()

X = data.drop('Disease', axis=1)  # Symptoms
y = data['Disease']  # Target label (Disease)


X = pd.get_dummies(X)  # One-hot encoding for categorical variables


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


import torch
import torch.nn as nn
import torch.optim as optim

class DiseaseClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(DiseaseClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Parameters
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(set(y))

model = DiseaseClassifier(input_size, hidden_size, num_classes)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


non_numeric_columns = X_train.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)


X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood Pressure        349 non-null    object
 8   Cholesterol Level     349 non-null    object
 9   Outcome Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB
None
Disease
Asthma                            23
Stroke                            16
Osteoporosis                      14
Hypertension                      10
Diabetes                          10
                                  ..
Autism Sp

In [32]:
import torch

# Save the model
torch.save(model.state_dict(), "disease_model.pth")


In [33]:
import os

# Find the directory
model_directory = os.path.abspath('disease_model_directory')
print(model_directory)


/content/CuraGen2024/CuraGen2024/disease_model_directory


In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Replace this with your actual model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model.save_pretrained('/content/disease_model_directory')
tokenizer.save_pretrained('/content/disease_model_directory')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('/content/disease_model_directory/tokenizer_config.json',
 '/content/disease_model_directory/special_tokens_map.json',
 '/content/disease_model_directory/vocab.txt',
 '/content/disease_model_directory/added_tokens.json',
 '/content/disease_model_directory/tokenizer.json')

In [35]:
!pip install streamlit



In [36]:
!pip install streamlit pyngrok


In [37]:
# %%writefile app.py

# import streamlit as st
# from transformers import pipeline

# # Streamlit app title
# st.title("Disease Prediction from Symptoms")

# # Text input for symptoms
# user_input = st.text_input("Enter your symptoms:")

# if user_input:
#     device_id = 0  # GPU device if available
#     pipe = pipeline("text-classification", model="/content/disease_model_directory", device=device_id)

#     # Get prediction
#     result = pipe(user_input)

#     # Display the result
#     st.write(f"Predicted Disease: {result[0]['label']}")


In [38]:
import streamlit as st
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Create a Streamlit app
st.title("Disease Prediction from Symptoms")

user_input = st.text_input("Enter your symptoms:")

if user_input:
    device_id = 0  # Use GPU if available, else CPU
    pipe = pipeline("text-classification", model="/content/disease_model_directory", device=device_id)

    result = pipe(user_input)
    st.write(f"Predicted Disease: {result[0]['label']}")

    # Print result for debugging (won't be seen in Streamlit UI)
    print(result)


In [39]:
import re

def validate_input(user_input):
    if not re.match("^[a-zA-Z0-9, ]*$", user_input):
        return False
    return True

if user_input:
    if validate_input(user_input):
        try:
            result = pipe(user_input)
            st.write(f"Predicted Disease: {result[0]['label']}")
        except Exception as e:
            st.write(f"An error occurred: {str(e)}")
    else:
        st.write("Please enter valid symptom descriptions.")

In [40]:
import streamlit as st
from transformers import pipeline
import torch

# Streamlit app title
st.title("Disease Prediction from Symptoms")

# User input for symptoms
user_input = st.text_input("Enter your symptoms:")

# Automatically choose GPU if available, otherwise fall back to CPU
device_id = 0 if torch.cuda.is_available() else -1

# Create the pipeline with the correct device
pipe = pipeline("text-classification", model="duxprajapati/symptom-disease-model", device=device_id)

# Process user input and display result
if user_input:
    result = pipe(user_input)
    st.write(f"Predicted Disease: {result[0]['label']}")

    # Print result for debugging (won't be seen in Streamlit UI)
    print(result)
